### trade based simulation

# data import using yfinance

## 라이브러리 임포트

In [162]:
!pip install yfinance
!pip install pandas-datareader

In [163]:
import pandas as pd
import numpy as np
from datetime import datetime
import math
import yfinance as yf
import pandas_datareader as pdr

In [164]:
current_date = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")

## 데이터 임포트

In [165]:
tickers = ["SPY", "QQQ", "GLD", "LQD", "EEM"]
risk_free_asset = "TB3MS"

In [166]:
divs = []
for ticker in tickers:
  div = yf.Ticker(ticker).dividends
  div.index = pd.to_datetime(div.index).strftime('%Y-%m')
  if not div.index.is_unique:
    div = div.groupby(div.index).sum()
  divs.append(div)

In [167]:
dividends_df = pd.concat(divs, axis=1)
dividends_df.columns = [["Dividend"] * len(tickers), tickers]
dividends_df.index.name = "Date"
dividends_df.sort_index(inplace=True)

In [168]:
dividends_df

Dividend                         
             SPY    QQQ GLD    LQD    EEM
Date                                     
1993-03    0.213    NaN NaN    NaN    NaN
1993-06    0.318    NaN NaN    NaN    NaN
1993-09    0.286    NaN NaN    NaN    NaN
1993-12    0.317    NaN NaN    NaN    NaN
1994-03    0.271    NaN NaN    NaN    NaN
...          ...    ...  ..    ...    ...
2021-08      NaN    NaN NaN  0.243    NaN
2021-09    1.428  0.414 NaN  0.234    NaN
2021-10      NaN    NaN NaN  0.255    NaN
2021-11      NaN    NaN NaN  0.262    NaN
2021-12    1.633  0.491 NaN  0.495  0.733

[250 rows x 5 columns]

In [169]:
tickers_df = yf.download(" ".join(tickers), period='max', interval='1mo')

[*********************100%***********************]  5 of 5 completed


In [170]:
tickers_df.index = pd.to_datetime(tickers_df.index).strftime('%Y-%m')

In [171]:
tickers_df.drop(columns=["Adj Close", "High", "Low", "Open", "Volume"], inplace=True)
tickers_df.dropna(how="all", inplace=True)

In [172]:
if not tickers_df.index.is_unique:
    tickers_df = tickers_df.groupby(tickers_df.index).last()

In [173]:
# tickers_df = pd.concat([tickers_df, dividends_df], axis=1)

In [174]:
# tickers_df.columns = tickers_df.columns.swaplevel(0,1)
# tickers_df.sort_index(axis=1, level=0, inplace=True)

In [175]:
tb3ms_df = pdr.DataReader(risk_free_asset, "fred", tickers_df.index[0], tickers_df.index[-1] )
tb3ms_df["RoR12"] = tb3ms_df[risk_free_asset].map(lambda x: x / 100)
tb3ms_df["CRoR"] = (1 + tb3ms_df["RoR12"] / 12).cumprod()
tb3ms_df.index = pd.to_datetime(tb3ms_df.index).strftime('%Y-%m')

In [176]:
tb3ms_df

,TB3MS,RoR12,CRoR
DATE,,,
1993-02,2.93,0.0293,1.002442
1993-03,2.95,0.0295,1.004906
1993-04,2.87,0.0287,1.007309
1993-05,2.96,0.0296,1.009794
1993-06,3.07,0.0307,1.012377
...,...,...,...
2021-08,0.05,0.0005,1.912575
2021-09,0.04,0.0004,1.912639
2021-10,0.05,0.0005,1.912718


In [177]:
tickers_df[("Close", risk_free_asset)] = tb3ms_df["CRoR"]

In [178]:
tickers_df

Close                                                          
               EEM         GLD         LQD         QQQ         SPY     TB3MS
Date                                                                        
1993-02        NaN         NaN         NaN         NaN   44.406250  1.002442
1993-03        NaN         NaN         NaN         NaN   45.187500  1.004906
1993-04        NaN         NaN         NaN         NaN   44.031250  1.007309
1993-05        NaN         NaN         NaN         NaN   45.218750  1.009794
1993-06        NaN         NaN         NaN         NaN   45.062500  1.012377
...            ...         ...         ...         ...         ...       ...
2021-09  50.380001  164.220001  133.029999  357.959991  429.140015  1.912639
2021-10  50.919998  166.649994  133.479996  386.109985  459.250000  1.912718
2021-11  48.840000  165.500000  133.059998  393.820007  455.559998  1.912798
2021-12  48.849998  170.960007  132.520004  397.850006  474.959991  1.912894
2022-01  47.290001  167.100006  127.699997  351.799988  441.950012       NaN

[348 rows x 6 columns]

# backtesting

In [179]:
exclude_period, look_back = 3, 12

In [259]:
ror_df = tickers_df.apply(lambda x: round((x.shift(exclude_period) - x.shift(look_back)) / x.shift(look_back), 4), axis=0)

In [260]:
ror_df.columns = ror_df.columns.map(lambda x: x[1])

In [261]:
# tickers_df = pd.concat([tickers_df, ror_df], axis=1)

In [262]:
ror_df

,EEM,GLD,LQD,QQQ,SPY,TB3MS
Date,,,,,,
1993-02,NaN,NaN,NaN,NaN,NaN,NaN
1993-03,NaN,NaN,NaN,NaN,NaN,NaN
1993-04,NaN,NaN,NaN,NaN,NaN,NaN
1993-05,NaN,NaN,NaN,NaN,NaN,NaN
1993-06,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2021-09,0.2509,-0.0649,-0.0026,0.2757,0.2782,0.0004
2021-10,0.1541,-0.0362,0.0170,0.3534,0.3429,0.0004
2021-11,0.0755,0.0181,-0.0230,0.2681,0.2472,0.0004


In [263]:
# ror_df.max(axis=1)

In [280]:
winners = ror_df.idxmax(axis=1)

In [281]:
winners[:look_back] = risk_free_asset

In [285]:
tickers_df["Buys"] = winners

In [292]:
tickers_df

Close                          ...                         Buys
               EEM         GLD         LQD  ...         SPY     TB3MS       
Date                                        ...                             
1993-02        NaN         NaN         NaN  ...   44.406250  1.002442  TB3MS
1993-03        NaN         NaN         NaN  ...   45.187500  1.004906  TB3MS
1993-04        NaN         NaN         NaN  ...   44.031250  1.007309  TB3MS
1993-05        NaN         NaN         NaN  ...   45.218750  1.009794  TB3MS
1993-06        NaN         NaN         NaN  ...   45.062500  1.012377  TB3MS
...            ...         ...         ...  ...         ...       ...    ...
2021-09  50.380001  164.220001  133.029999  ...  429.140015  1.912639    SPY
2021-10  50.919998  166.649994  133.479996  ...  459.250000  1.912718    QQQ
2021-11  48.840000  165.500000  133.059998  ...  455.559998  1.912798    QQQ
2021-12  48.849998  170.960007  132.520004  ...  474.959991  1.912894    SPY
2022-01  47.290001  167.100006  127.699997  ...  441.950012       NaN    SPY

[348 rows x 7 columns]

In [336]:
idx, cols = pd.factorize(winners)

In [338]:
cols = cols.map(lambda x: ("Close", x))

In [343]:
tickers_df.reindex(cols, axis=1).to_numpy()[np.arange(len(idx)), idx]

array([  1.00244167,   1.004906  ,   1.0073094 ,   1.0097941 ,
         1.01237749,   1.01494218,   1.01749645,   1.0199978 ,
         1.02256479,   1.02520642,   1.02782069,   1.03037311,
        46.8125    ,  44.59375   ,  45.09375   ,  45.8125    ,
         1.04653863,   1.05031489,   1.05423607,   1.05829487,
         1.06266034,  45.59375   ,   1.07232584,   1.07742833,
         1.08260896,   1.08777842,  51.59375   ,  53.640625  ,
        54.40625   ,  56.15625   ,  56.40625   ,  58.484375  ,
        58.3125    ,  60.90625   ,  61.484375  ,  63.671875  ,
        63.875     ,  64.6875    ,  65.390625  ,  66.875     ,
        67.109375  ,  64.09375   ,  65.328125  ,  68.625     ,
        70.84375   ,  76.015625  ,  73.84375   ,  78.40625   ,
        79.15625   ,  75.375     ,  80.09375   ,  85.15625   ,
        88.3125    ,  95.3125    ,  90.375     ,  94.375     ,
        92.0625    ,  95.625     ,  97.0625    ,  98.3125    ,
       105.125     , 109.9375    , 111.34375   , 109.03